In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import datasets, layers, models, Sequential, backend, callbacks

try:
    from scikeras.wrappers import KerasClassifier, KerasRegressor
except ImportError:
    !python -m pip install scikeras
    from scikeras.wrappers import KerasClassifier, KerasRegressor
  
import cv2
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(linewidth=100000, precision=3)
import sys
import pandas as pd

from skimage import data, io, filters
from skimage.transform import resize
from skimage.transform import rotate

from sklearn.model_selection import GridSearchCV, KFold
import datetime
from tqdm.keras import TqdmCallback

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
(img_train, label_train), (img_test, label_test) = tfds.as_numpy(tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

Num GPUs Available:  4


2022-03-16 02:41:52.090756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 02:41:52.090982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 02:41:52.091147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 02:41:52.091308: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 02:41:52.116962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

#Using a NNet for classification

### Prelude
- The data is in the form of a 28x28 matrix that contains the greyscale pixel of an object. To use it for a Neural network, we will apply convolution to it  and then pool it into a smaller matrix as the data procesing step i.e. we will use a CNN. In addition to that, we normalize the image's pixel values to be a number between 0 and 1 by dividing by 255

In [3]:
img_train = img_train / 255
img_test = img_test / 255

#### CNN with Maxpooling

In [4]:
%%script false --no-raise-error
model = models.Sequential() # Create the model
model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28 , 1))) # add the first layer to convole the input matrix using a 3x3 kernal with a relu activation to prevent linearity
model.add(layers.MaxPooling2D(pool_size = (2, 2))) #pool to decrease dimension of image
model.add(layers.Conv2D(56, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2, 2))) #pool
model.add(layers.Conv2D(112, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(112, activation='relu'))
model.add(layers.Dense(10))

In [5]:
%%script false --no-raise-error
model.summary()

In [6]:
%%script false --no-raise-error
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(img_train, label_train, epochs=10, 
                    validation_data=(img_test, label_test))

In [7]:
%%script false --no-raise-error
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(img_test,  label_test, verbose=2)

#### CNN with avgerage pooling

In [8]:
%%script false --no-raise-error
model = models.Sequential() # Create the model
model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28 , 1))) # add the first layer to convole the input matrix using a 3x3 kernal with a relu activation to prevent linearity
model.add(layers.AveragePooling2D(pool_size = (2, 2))) #pool to decrease dimension of image
model.add(layers.Conv2D(56, (3, 3), activation='relu'))
model.add(layers.AveragePooling2D(pool_size = (2, 2))) #pool
model.add(layers.Conv2D(112, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(112, activation='relu'))
model.add(layers.Dense(10))

In [9]:
%%script false --no-raise-error
model.summary()

In [10]:
%%script false --no-raise-error
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(img_train, label_train, epochs=10, 
                    validation_data=(img_test, label_test))

In [11]:
%%script false --no-raise-error
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(img_test,  label_test, verbose=2)
backend.clear_session()

### Data Augmentation

In [12]:
augmented_data = []
augmented_data_ys = []
for i,img in enumerate(img_train):
    #rotate 90 degrees
    #augmented_data.append(rotate(img,90))
    #augmented_data_ys.append(label_train[i])

    #rotate 180 degrees
    #augmented_data.append(rotate(img,180))
    #augmented_data_ys.append(label_train[i])

    #rotate 270 degrees
    #augmented_data.append(rotate(img,270))
    #augmented_data_ys.append(label_train[i])

    #flip left to right
    augmented_data.append(np.fliplr(img))
    augmented_data_ys.append(label_train[i])

In [13]:
augmented_data = np.array(augmented_data)
augmented_data_ys = np.array(augmented_data_ys)
img_train = np.concatenate((img_train,augmented_data))
print(img_train.shape)
label_train = np.concatenate((label_train,augmented_data_ys))
print(label_train.shape)

(120000, 28, 28, 1)
(120000,)


### CNN with augmented data and GridSearchCV

In [ ]:
# Function to create model, required for KerasClassifier
def create_cnn_model( num_hidden_layers=0, num_hidden_nodes=0, dropout = 0, activation='relu' , optimizer='adam'):
    backend.clear_session()
    model = models.Sequential() # Create the model
    model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28 , 1))) # add the first layer to convole the input matrix using a 3x3 kernal with a relu activation to prevent linearity
    model.add(layers.MaxPooling2D(pool_size = (2, 2))) #pool to decrease dimension of image
    model.add(layers.Conv2D(56, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size = (2, 2))) #pool
    model.add(layers.Conv2D(112, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(112, activation=activation))    
    
    for i in range(num_hidden_layers):
        tf.keras.layers.Dropout(dropout)
        if num_hidden_nodes == 0:
            model.add(layers.Dense(112, activation=activation))
        else:
            model.add(layers.Dense(num_hidden_nodes, activation=activation))

    model.add(layers.Dense(10))
    model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    #model.summary()
    return model

hLayers = [1, 2, 3, 4, 5]
hNodes = [100, 150, 200, 250]

num_folds = 5
num_epochs= 20

cv_results = {}
cv_results['param_layers'] = []
cv_results['param_nodes'] = []
cv_results['mean_test_score'] = []
cv_results['std_test_score'] = []
cv_results['test_score'] = []
for i in range(num_folds):
    cv_results[f'split{i}_test_score'] = []


for layer in hLayers:
    for node in hNodes:
        print(f'Currently on Layer {layer}, node {node}')
        i = 0
        kfold = KFold(num_folds, shuffle=True, random_state=1)
        accs = []
        cv_results['param_layers'].append(layer)
        cv_results['param_nodes'].append(node)
        for train_ix, test_ix in kfold.split(img_train):
            print(f'Fold {i+1}/{num_folds}, params layers = {layer}, nodes = {node}')
            model = create_cnn_model(layer, node)
            trainX, trainY = img_train[train_ix], label_train[train_ix]
            testX, testY = img_train[test_ix], label_train[test_ix]
            history = model.fit(trainX, trainY, epochs=num_epochs, 
                        validation_data=(testX, testY), verbose=0, callbacks=[TqdmCallback(verbose=1)])
            test_loss, test_acc = model.evaluate(testX,  testY, verbose=2)
            
            cv_results[f'split{i}_test_score'].append(test_acc)
            accs.append(test_acc)
            i += 1
            del model
        print(f'Mean CV score + stddev: {np.mean(accs)} +/- {np.std(accs)}')
        cv_results['mean_test_score'].append(np.mean(accs))
        cv_results['std_test_score'].append(np.std(accs))
        model = create_cnn_model(layer, node)
        history = model.fit(img_train, label_train, epochs=num_epochs, 
                        validation_data=(img_test, label_test), verbose=0, callbacks=[TqdmCallback(verbose=1)])
        test_loss, test_acc = model.evaluate(img_test, label_test, verbose=2)
        print(f'Accuracy on test set: {test_acc}')
        cv_results['test_score'].append(test_acc)
        del model
        


Currently on Layer 1, node 100
Fold 1/5, params layers = 1, nodes = 100


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

750/750 - 1s - loss: 0.3569 - accuracy: 0.9189 - 1s/epoch - 1ms/step
Fold 2/5, params layers = 1, nodes = 100


0epoch [00:00, ?epoch/s]

...

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

750/750 - 1s - loss: 0.2724 - accuracy: 0.9240 - 1s/epoch - 2ms/step
Fold 2/5, params layers = 5, nodes = 150


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [ ]:
print(cv_results)
results_df = pd.DataFrame.from_dict(cv_results)
print(results_df)
results_df.to_pickle("cnn.pkl")

In [ ]:
%%script false --no-raise-error
# Function to create model, required for KerasClassifier
def create_model( activation='relu' , optimizer='adam'):
    backend.clear_session()
    model = models.Sequential() # Create the model
    model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28 , 1))) # add the first layer to convole the input matrix using a 3x3 kernal with a relu activation to prevent linearity
    model.add(layers.MaxPooling2D(pool_size = (2, 2))) #pool to decrease dimension of image
    model.add(layers.Conv2D(56, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size = (2, 2))) #pool
    model.add(layers.Conv2D(112, (3, 3), activation='relu'))


    model.add(layers.Flatten())
    model.add(layers.Dense(112, activation=activation))
    model.add(layers.Dense(10))
    model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    #model.summary()
    return model

model = KerasClassifier(model=create_model, activation='relu' , optimizer='adam' ,verbose=0)
# define the grid search parameters
#activation = ['relu', 'tanh', 'sigmoid']
optimizer = ['SGD', 'Adagrad',  'Adam']
#convnet are sensitive to batch_size and epochs so do grid search on those too
#batch_size = [5, 10, 15] #cant try more due to limited ram
#epochs = [10, 20, 30]
#param_grid = dict( activation=activation, optimizer=optimizer , epochs=epochs)
param_grid = dict( optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=5, verbose=10)
print(grid)

grid_result = grid.fit(img_train, label_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
%%script false --no-raise-error
print(grid.best_params_)
print(grid.best_index_)
print(grid.best_score_)
results_df = pd.DataFrame.from_dict(grid.cv_results_)
print(results_df)
results_df.to_pickle("cnn.pkl")